In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import scipy.misc
import _pickle as cPickle

%matplotlib inline

In [2]:
tf.enable_eager_execution()

In [ ]:
path = './cifar-10/'

In [ ]:
def unzip(filename):
    fo=open(filename,'rb')
    data=cPickle.load(fo, encoding='latin1')
    return data

In [ ]:
train=unzip(path+'train_data')
validate=unzip(path+'validate_data')
test=unzip(path+'test_data')

In [8]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
#     # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
#     X_train=np.asarray(train['data'],dtype=np.float32)
#     X_train=X_train.reshape(X_train.shape[0], 32, 32, 3)
#     y_train=np.asarray(train['labels'],dtype=np.int32)
#     X_test=np.asarray(test['data'],dtype=np.float32)
#     X_test=X_test.reshape(X_test.shape[0], 32, 32, 3)
#     y_test=np.asarray(test['labels'],dtype=np.int32)
    
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [9]:
train_examples_with_label = {}
valid_examples_with_label = {}
test_examples_with_label = {}

In [10]:
for i in range(10):
    train_examples_with_label[i] = np.array([index for index, value in enumerate(y_train) if value == i])
    test_examples_with_label[i] = np.array([index for index, value in enumerate(y_test) if value == i])
    valid_examples_with_label[i] = np.array([index for index, value in enumerate(y_val) if value == i])

In [11]:
training_set_size = 100
test_set_size = 100
valid_set_size = 50

In [12]:
train_labels = []
test_labels = []
valid_labels = []

In [13]:
import random
for i in range(10):
    train_labels = train_labels + random.sample(list(train_examples_with_label[i]), int(training_set_size/10))
    test_labels = test_labels + random.sample(list(test_examples_with_label[i]), int(test_set_size/10))
    valid_labels = valid_labels + random.sample(list(valid_examples_with_label[i]), int(valid_set_size/10))
random.shuffle(train_labels)
random.shuffle(test_labels)
random.shuffle(valid_labels)

In [14]:
X_train = X_train[train_labels]
y_train = y_train[train_labels]
X_test = X_test[test_labels]
y_test = y_test[test_labels]
X_val = X_val[valid_labels]
y_val = y_val[valid_labels]

In [15]:
def resize_datasets(X):
    X_new = []
    for i in range(X.shape[0]):
        if(i % 100 == 0):
            print(i)
        img = scipy.ndimage.interpolation.zoom(X[i], (8.0, 8.0, 1.0))
        X_new.append(img)
    X_new = np.asarray(X_new, dtype=np.float32)
    return X_new.reshape([X.shape[0], 256, 256, 3])

In [ ]:
print('Resizing Validation...')
X_val = resize_datasets(X_val)

In [ ]:
print('Resizing Test...')
X_test = resize_datasets(X_test)

In [16]:
print('Resizing Training...')
X_train = resize_datasets(X_train)

Resizing Training...
0


In [17]:
X_train.shape

(100, 256, 256, 3)

In [ ]:
import os, pickle
file_path = "train_dataset.pkl"
max_bytes = 2**31 - 1

In [ ]:
## write
bytes_out = pickle.dumps(X_train)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

In [7]:
import pickle
pickle_in = open("dataset.pickle","rb")
X_train = pickle.load(pickle_in)
y_train = pickle.load(pickle_in)
X_test = pickle.load(pickle_in)
y_test = pickle.load(pickle_in)
X_val = pickle.load(pickle_in)
y_val = pickle.load(pickle_in)

EOFError: Ran out of input

In [ ]:
## read
bytes_in = bytearray(0)
input_size = os.path.getsize(file_path)
with open(file_path, 'rb') as f_in:
    for _ in range(0, input_size, max_bytes):
        bytes_in += f_in.read(max_bytes)
X_train = pickle.loads(bytes_in)

In [ ]:
X_train.shape

In [6]:
import pickle
pickle_in = open("dataset.pickle","rb")
X_train = pickle.load(pickle_in)
y_train = pickle.load(pickle_in)
X_test = pickle.load(pickle_in)
y_test = pickle.load(pickle_in)
X_val = pickle.load(pickle_in)
y_val = pickle.load(pickle_in)

EOFError: Ran out of input

In [19]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [20]:
def flatten(x):
    """    
    Input:
    - TensorFlow Tensor of shape (N, D1, ..., DM)
    
    Output:
    - TensorFlow Tensor of shape (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [ ]:
inp = tf.get_variable(name = 'input', shape=[1,256,256,3], dtype=tf.float32)

In [21]:
#globals used in the assignment
IMG_SHAPE = [256, 256, 3] # height, width, channels
NUM_CLASSES = 10

# training hyperparameters
LEARNING_RATE = 1e-2
MOMENTUM = 0.9
BATCH_SIZE = 16
EPOCHS = 10
DISPLAY_STEP = 1
VALIDATION_STEP = 1000
SAVE_STEP = 1000
CKPT_PATH = './ckpt'
SUMMARY_PATH = './summary'

# net architecture hyperparamaters
LAMBDA = 5e-4 #for weight decay
DROPOUT = 0.5

# test hyper parameters
K_PATCHES = 5
TOP_K = 5

In [22]:
class AlexNet(tfe.Network):
    def __init__(self, training):
        super(AlexNet, self).__init__()
        self.training = training
        
        # convolutional layers
        conv_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        self.conv1 = self.track_layer(
            tf.layers.Conv2D(
                96,
                11,
                4,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool1 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv2 = self.track_layer(
            tf.layers.Conv2D(
                256,
                5,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool2 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv3 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv4 = self.track_layer(
            tf.layers.Conv2D(
                384,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))

        self.conv5 = self.track_layer(
            tf.layers.Conv2D(
                256,
                3,
                1,
                'SAME',
                activation=tf.nn.relu,
                kernel_initializer=conv_init))
        self.pool5 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        # fully connected layers

        fc_init = tf.contrib.layers.xavier_initializer()

        self.fc1 = self.track_layer(
            tf.layers.Dense(
                4096, activation=tf.nn.relu, kernel_initializer=fc_init))
        self.drop1 = self.track_layer(tf.layers.Dropout(DROPOUT))

        self.fc2 = self.track_layer(
            tf.layers.Dense(
                4096, activation=tf.nn.relu, kernel_initializer=fc_init))
        self.drop2 = self.track_layer(tf.layers.Dropout(DROPOUT))
        
        self.out = self.track_layer(
            tf.layers.Dense(NUM_CLASSES, kernel_initializer=fc_init))

    def call(self, x):
        """ Function that executes the model """
        output = self.conv1(x)
#         print("After conv1 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
#         print("After normalisation1 : " + str(output.numpy().shape))
        output = self.pool1(output)
#         print("After pool1 : " + str(output.numpy().shape))

        output = self.conv2(output)
#         print("After conv2 : " + str(output.numpy().shape))
        output = tf.nn.lrn(
            output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
#         print("After normalisation2 : " + str(output.numpy().shape))
        output = self.pool2(output)
#         print("After pool2 : " + str(output.numpy().shape))

        output = self.conv3(output)
#         print("After conv3 : " + str(output.numpy().shape))

        output = self.conv4(output)
#         print("After conv4 : " + str(output.numpy().shape))
        conv4_output = output

        output = self.conv5(output)
#         print("After conv5 : " + str(output.numpy().shape))
        conv5_output = output
        
        output = self.pool5(output)
#         print("After pool5 : " + str(output.numpy().shape))

        output = tf.layers.flatten(output)
#         print("After flattening : " + str(output.numpy().shape))

        output = self.fc1(output)
#         print("After fc1 : " + str(output.numpy().shape))
        if self.training:
            output = self.drop1(output)
#             print("After drop1 : " + str(output.numpy().shape))

        output = self.fc2(output)
#         print("After fc2 : " + str(output.numpy().shape))
        if self.training:
            output = self.drop2(output)
#             print("After drop2 : " + str(output.numpy().shape))
        
        output = self.out(output)
        
        return output

In [23]:
def loss(net, mode, x, y):
    """
    Computes the loss for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the loss between the predictions on the images and the groundtruths
    """
    pred = net(x)
    y = tf.one_hot(y, NUM_CLASSES)

    loss_value = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)
    weight_decay = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in net.variables]))

    total_loss = loss_value + weight_decay

    tf.contrib.summary.scalar(mode + '/loss', total_loss)

    return total_loss

In [24]:
def accuracy(net, mode, x, y):
    """
    Computes the accuracy for a given batch of examples
    Args:
        mode, string 'train' or 'val'
        x, tf tensor representing a batch of images
        y, tf tensor representing a batch of labels
    Returns:
        the accuracy of the predictions on the images and the groundtruths
    """
    pred = tf.nn.softmax(net(x))
    y = tf.one_hot(y, NUM_CLASSES)

    accuracy_value = tf.reduce_sum(
                tf.cast(
                    tf.equal(
                        tf.argmax(pred, axis=1, output_type=tf.int64),
                        tf.argmax(y, axis=1, output_type=tf.int64)
                    ),
                    dtype=tf.float32
                ) 
            ) / float(pred.shape[0].value)

    tf.contrib.summary.scalar(mode +'/accuracy', accuracy_value)

    return accuracy_value

In [25]:
train_dset = Dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=valid_set_size, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=test_set_size, shuffle=True)

In [26]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
#     print(y)
    if t > 5: break

0 (16, 256, 256, 3) (16,)
1 (16, 256, 256, 3) (16,)
2 (16, 256, 256, 3) (16,)
3 (16, 256, 256, 3) (16,)
4 (16, 256, 256, 3) (16,)
5 (16, 256, 256, 3) (16,)
6 (4, 256, 256, 3) (4,)


In [27]:
DISPLAY_STEP = 1
VALIDATION_STEP = 5
SAVE_STEP = 10

In [28]:
#used for printing the log with the timestamp
logging = tf.logging
logging.set_verbosity(logging.INFO)
def log_msg(msg):
       logging.info(f'{time.ctime()}: {msg}') 

In [29]:
import time
def train(net):
    """
    Training procedure
    """
    start_time = time.time()
    step_time = 0.0
    
    writer = tf.contrib.summary.create_summary_file_writer(SUMMARY_PATH)
#     optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
    optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    global_step = tf.train.get_or_create_global_step()
    epoch = tfe.Variable(0, name='epoch', dtype=tf.float32, trainable=False)
    all_variables = (net.variables + optimizer.variables() + [global_step] + [epoch])
    
    with writer.as_default():
        with tf.contrib.summary.record_summaries_every_n_global_steps(DISPLAY_STEP):
            for e in range(int(epoch.numpy()), EPOCHS):
                tf.assign(epoch, e)
                for (batch_i, (images, labels)) in enumerate(train_dset):
                    global_step = tf.train.get_global_step()
                    step = global_step.numpy() + 1
                    images = tf.to_float(images)
                    step_start_time = int(round(time.time() * 1000))
                    
                    optimizer.minimize(lambda: loss(net, 'train', images, labels), global_step=global_step)

                    step_end_time = int(round(time.time() * 1000))
                    step_time += step_end_time - step_start_time

                    if (step % DISPLAY_STEP) == 0:
                        l = loss(net, 'train', images, labels)
                        a = accuracy(net, 'train', images, labels).numpy()
                        log_msg('Epoch: {:03d} Step/Batch: {:09d} Step mean time: {:04d}ms \nLoss: {:.7f} Training accuracy: {:.4f}'.format(e, step, int(step_time / step), l, a))

#                     if (step % VALIDATION_STEP) == 0:
#                         for (_, (val_images, val_labels)) in enumerate(val_dset):
# #                             val_images, val_labels = tfe.Iterator(val_dset).next()
#                             l = loss(net, 'val', val_images, val_labels)
#                             a = accuracy(net, 'val', val_images, val_labels).numpy()
#                             int_time = time.time() - start_time
#                             log_msg('Elapsed time: {:04d}ms --- Loss: {:.7f} Validation accuracy: {:.4f}'.format(int(int_time), l, a))

#                     if (step % SAVE_STEP) == 0:
#                         tfe.Saver(net.variables).save(os.path.join(CKPT_PATH, 'net.ckpt'), global_step=global_step)
#                         log_msg('Variables saved')

In [30]:
alexnet = AlexNet(training=True)

Instructions for updating:
Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).

Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but

In [31]:
if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
train(alexnet)

INFO:tensorflow:Mon Nov 26 15:52:40 2018: Epoch: 000 Step/Batch: 000000001 Step mean time: 3799ms 
Loss: 7665465.5000000 Training accuracy: 0.1875
INFO:tensorflow:Mon Nov 26 15:52:44 2018: Epoch: 000 Step/Batch: 000000002 Step mean time: 3185ms 
Loss: 363.0084839 Training accuracy: 0.1875
INFO:tensorflow:Mon Nov 26 15:52:48 2018: Epoch: 000 Step/Batch: 000000003 Step mean time: 2957ms 
Loss: 61.6835480 Training accuracy: 0.0000
INFO:tensorflow:Mon Nov 26 15:52:52 2018: Epoch: 000 Step/Batch: 000000004 Step mean time: 2849ms 
Loss: 4718.1572266 Training accuracy: 0.1875
INFO:tensorflow:Mon Nov 26 15:52:56 2018: Epoch: 000 Step/Batch: 000000005 Step mean time: 2783ms 
Loss: 377.3768921 Training accuracy: 0.0625
INFO:tensorflow:Mon Nov 26 15:52:59 2018: Epoch: 000 Step/Batch: 000000006 Step mean time: 2740ms 
Loss: 22544.8417969 Training accuracy: 0.1875
INFO:tensorflow:Mon Nov 26 15:53:01 2018: Epoch: 000 Step/Batch: 000000007 Step mean time: 2500ms 
Loss: 329.3940735 Training accuracy: 

INFO:tensorflow:Mon Nov 26 15:56:02 2018: Epoch: 008 Step/Batch: 000000058 Step mean time: 2376ms 
Loss: 24.8975620 Training accuracy: 0.1250
INFO:tensorflow:Mon Nov 26 15:56:06 2018: Epoch: 008 Step/Batch: 000000059 Step mean time: 2378ms 
Loss: 25.0748138 Training accuracy: 0.1875
INFO:tensorflow:Mon Nov 26 15:56:09 2018: Epoch: 008 Step/Batch: 000000060 Step mean time: 2380ms 
Loss: 138.1916809 Training accuracy: 0.2500
INFO:tensorflow:Mon Nov 26 15:56:13 2018: Epoch: 008 Step/Batch: 000000061 Step mean time: 2382ms 
Loss: 25.2194157 Training accuracy: 0.0625
INFO:tensorflow:Mon Nov 26 15:56:17 2018: Epoch: 008 Step/Batch: 000000062 Step mean time: 2384ms 
Loss: 25.3128147 Training accuracy: 0.0625
INFO:tensorflow:Mon Nov 26 15:56:18 2018: Epoch: 008 Step/Batch: 000000063 Step mean time: 2362ms 
Loss: 25.5467224 Training accuracy: 0.2500
INFO:tensorflow:Mon Nov 26 15:56:22 2018: Epoch: 009 Step/Batch: 000000064 Step mean time: 2366ms 
Loss: 25.5251312 Training accuracy: 0.1250
INFO:

In [ ]:
for (_, (test_images, test_labels)) in enumerate(test_dset):
    l = loss(alexnet, 'val', test_images, test_labels)
    a = accuracy(alexnet, 'val', test_images, test_labels).numpy()
log_msg('Final Test Loss: {:.7f} Test accuracy: {:.4f}'.format(l, a))